## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
#from config import g_key
import os
from dotenv import load_dotenv
load_dotenv()
g_key = os.getenv("g_key")

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,3,Nishihara,JP,80.26,moderate rain,26.1842,127.7558,Minshuku Agaihama
1,4,Lata,PT,87.49,clear sky,40.1629,-8.3327,Casa Maquia
2,8,Banda Aceh,ID,80.53,overcast clouds,5.5577,95.3222,OYO 884 Rumoh PMI Hotel
3,12,Butaritari,KI,81.18,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
4,14,Ribeira Grande,PT,77.40,broken clouds,38.5167,-28.7000,Quinta da Meia Eira
5,15,Balikpapan,ID,76.19,heavy intensity rain,-1.2675,116.8289,Swiss-Belhotel Balikpapan
6,16,Georgetown,MY,82.33,few clouds,5.4112,100.3354,Cititel Penang
7,18,Saint-Georges,GD,85.68,few clouds,12.0564,-61.7485,Radisson Grenada Beach Resort
8,24,Boa Vista,BR,75.18,light rain,2.8197,-60.6733,Aipana Plaza Hotel
9,31,Buala,SB,79.34,scattered clouds,-8.1450,159.5921,Maringe Lagoon Lodge


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.
max_temp=vacation_df["Max Temp"]
fig=gmaps.figure(center=(30.0,31.0),zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations,weights=max_temp,
                                 dissipating=False,
                              max_intensity=300, point_radius=4)
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [11]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"]=="Broome"]
vacation_end = vacation_df.loc[vacation_df["City"]=="Broome"]
vacation_stop1 = vacation_df.loc[vacation_df["City"]=="Jamestown"]
vacation_stop2 = vacation_df.loc[vacation_df["City"]=="Fairmont"]
vacation_stop3 = vacation_df.loc[vacation_df["City"]=="Hamilton"]
vacation_stop1

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
27,70,Jamestown,US,75.52,broken clouds,42.097,-79.2353,Hampton Inn & Suites Jamestown


In [13]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0],vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0],vacation_end["Lng"].to_numpy()[0]
stop1 = vacation_stop1["Lat"].to_numpy()[0],vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0],vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0],vacation_stop3["Lng"].to_numpy()[0]

In [15]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
#waypoints=[stop1,stop2,stop3]
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
direction=gmaps.directions_layer(start=start,end=end,
                                 waypoints=[stop1,stop2,stop3],
                                 travel_mode="DRIVING")
fig.add_layer(direction)
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,221,Broome,US,81.63,broken clouds,42.2506,-75.8330,Chenango Valley State Park
1,70,Jamestown,US,75.52,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
2,417,Fairmont,US,81.52,broken clouds,39.4851,-80.1426,Clarion Inn
3,175,Hamilton,US,84.25,clear sky,39.1834,-84.5333,Six Acres Bed & Breakfast


In [17]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map.
fig=gmaps.figure()
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))